In [1]:
import sys
sys.path.append('/home/wangs/rs/lib')
import ff
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from multiprocessing import Pool
from tqdm import tqdm
import statsmodels.api as sm
from datetime import datetime
import mplfinance as mpf
from functools import partial
import matplotlib as mpl

In [2]:
def stock_mean(n):
    try:
        stock_mean = (ff.read_min(n).iloc[:,:4].loc[data_all.index[0]:data_all.index[-1]]).stack().rolling(window = 20).mean().unstack()['high'].round(8)
        # stock_mean = data.stack().rolling(window = 20).mean().unstack()['high']
    except:
        return pd.Series(np.nan,index = data_all.index).values
    return pd.Series(stock_mean,index = data_all.index).values
def stock_std(n):
    try:
        stock_std = (ff.read_min(n).iloc[:,:4].loc[data_all.index[0]:data_all.index[-1]]).stack().rolling(window = 20).std().unstack()['high'].round(8)
        # stock_std = data.stack().rolling(window = 20).std().unstack()['high']
    except:
        return pd.Series(np.nan,index = data_all.index).values
    return pd.Series(stock_std,index = data_all.index).values

def volume_min(n):
    try:
        test = ff.read_min(n).iloc[:,4].loc[data_all.index[0]:data_all.index[-1]]
        volume_daily = test.rolling(window = 240).sum()
        volume_daily = volume_daily[volume_daily.index.str.endswith('15:00:00')]
        result = (test / np.repeat(volume_daily.values,240)).round(8)
        return pd.Series(result,index = data_all.index).values
    except:
        return pd.Series(np.nan,index = data_all.index).values

def stock_corr(n):
    corr = []
    for i in range(int(data_all.shape[0]/240)):
        corr.append(np.corrcoef(single_stock_volume[n][i],relate_stock_volume[n][i])[0,1])
    return corr

In [11]:
data_all = pd.read_pickle('/mydata2/wangs/data/fmins/close.pk')
data_all = data_all.loc[data_all.index[-22*240:]]
single_stock_volume = np.empty([data_all.shape[1],int(data_all.shape[0]/240),240]) 
relate_stock_volume = np.empty([data_all.shape[1],int(data_all.shape[0]/240),240]) 

In [12]:
global single_stock_volume
global relate_stock_volume
with Pool(24) as p:
    res_lst = list(tqdm(p.imap(stock_mean,data_all.columns),total = len(data_all.columns)))
data_mean = np.vstack(res_lst)
with Pool(24) as p:
    res_lst = list(tqdm(p.imap(stock_std,data_all.columns),total = len(data_all.columns)))
data_std = np.vstack(res_lst)
up_status = data_all.values.T > (data_std + data_mean)
down_status = data_all.values.T < (data_mean - data_std)
mid_status = (data_all.values.T <= (data_std + data_mean)) * (data_all.values.T >= (data_mean - data_std))
with Pool(24) as p:
    res_lst = list(tqdm(p.imap(volume_min,data_all.columns),total = len(data_all.columns)))
data_volume_min = np.vstack(res_lst)
Cor_up_stock_volume = data_volume_min * up_status
Cor_down_stock_volume = data_volume_min * down_status
Cor_mid_stock_volume = data_volume_min * mid_status
single_stock_volume = np.reshape(data_volume_min,(data_volume_min.shape[0],-1,240))
relate_stock_volume = np.reshape((up_status * np.nansum(Cor_up_stock_volume,axis = 0) + mid_status * np.nansum(Cor_mid_stock_volume,axis = 0) + down_status* np.nansum(Cor_down_stock_volume,axis = 0)),(data_volume_min.shape[0],-1,240))
with Pool(24) as p:
    res_lst = list(tqdm(p.imap(stock_corr,range(data_volume_min.shape[0])),total = len(range(data_volume_min.shape[0]))))
corr_result = np.vstack(res_lst)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5246/5246 [00:00<00:00, 16036.62it/s]


In [117]:
def check(new_f, org_f):
    syms_set = sorted(list((set(new_f.index) & set(org_f.index))))
    dates_set = sorted(list((set(new_f.columns) & set(org_f.columns))))
    # ident = new_f.loc[syms_set,dates_set].equals(org_f.loc[syms_set,dates_set])
    ident = (new_f.loc[syms_set,dates_set]-(org_f.loc[syms_set,dates_set]))>0.001
    if (ident.sum().values == 0):
        return True
    else:
        display(new_f.loc[syms_set,dates_set].compare(org_f.loc[syms_set,dates_set]))
        return False

In [118]:
factor = pd.DataFrame(ff.rolling_window(corr_result,window = 20).mean(axis = -1) + ff.rolling_window(corr_result,window = 20).std(axis = -1))/2
factor.index = data_all.columns
factor.columns = np.unique(list(pd.to_datetime(data_all.index).strftime("%Y%m%d")))[19:]
factor = factor.iloc[:,1:]
factor
his_synergy = ff.read('synergy').iloc[:,-1:].to_dict()
new_synergy = factor.to_dict()
his_synergy.update(new_synergy)
his_synergy = pd.DataFrame(his_synergy)
assert check(ff.read('synergy'),his_synergy)

In [13]:
ff.save('synergy',factor)

In [20]:
pd.concat([ff.read_min('000001.SZ')['close'],ff.read_min('000001.SZ')['close']],axis = 1)

,close,close
2020-01-02 09:31:00,16.70,16.70
2020-01-02 09:32:00,16.83,16.83
2020-01-02 09:33:00,16.77,16.77
2020-01-02 09:34:00,16.80,16.80
2020-01-02 09:35:00,16.80,16.80
...,...,...
2024-05-23 14:56:00,11.39,11.39
2024-05-23 14:57:00,11.39,11.39
2024-05-23 14:58:00,11.39,11.39
2024-05-23 14:59:00,11.39,11.39


In [22]:
data_all.columns

Index(['000001.SZ', '000002.SZ', '000004.SZ', '000005.SZ', '000006.SZ',
       '000007.SZ', '000008.SZ', '000009.SZ', '000010.SZ', '000011.SZ',
       ...
       '600891.SH', '600896.SH', '600978.SH', '601258.SH', '601558.SH',
       '603157.SH', '603555.SH', '603996.SH', '688086.SH', '688555.SH'],
      dtype='object', length=5246)

In [ ]:
result = ff.read_min('000001.SZ')['close']
for i in list(data_all.columns):
    result = pd.concat([result,ff.read_min(i)['close']],axis = 1)

In [28]:
result

,close,close,close,close,close,close,close,close,close,close,close
2020-01-02 09:31:00,16.70,16.70,33.01,22.85,3.13,5.42,9.55,3.66,6.26,3.40,9.49
2020-01-02 09:32:00,16.83,16.83,33.10,22.78,3.14,5.41,9.48,3.65,6.21,3.41,9.45
2020-01-02 09:33:00,16.77,16.77,33.18,22.70,3.14,5.42,9.46,3.65,6.27,3.42,9.44
2020-01-02 09:34:00,16.80,16.80,33.44,22.49,3.14,5.42,9.50,3.64,6.26,3.42,9.45
2020-01-02 09:35:00,16.80,16.80,33.49,22.43,3.15,5.44,9.50,3.64,6.27,3.41,9.46
...,...,...,...,...,...,...,...,...,...,...,...
2024-05-23 14:56:00,11.39,11.39,9.53,11.36,NaN,4.13,4.62,2.20,10.15,1.90,9.01
2024-05-23 14:57:00,11.39,11.39,9.52,11.39,NaN,4.14,4.62,2.20,10.16,1.90,9.00
2024-05-23 14:58:00,11.39,11.39,9.53,11.39,NaN,4.14,4.62,2.20,10.16,1.90,9.01
2024-05-23 14:59:00,11.39,11.39,9.53,11.39,NaN,4.14,4.62,2.20,10.16,1.90,9.01


In [27]:
data_all = pd.read_pickle('/mydata2/wangs/data/fmins/close.pk')
data_all

,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,...,600891.SH,600896.SH,600978.SH,601258.SH,601558.SH,603157.SH,603555.SH,603996.SH,688086.SH,688555.SH
2020-01-02 09:31:00,16.70,33.01,22.85,3.13,5.42,9.55,3.66,6.26,3.40,9.49,...,1.49,4.28,3.00,1.42,1.10,5.80,5.91,4.91,NaN,NaN
2020-01-02 09:32:00,16.83,33.10,22.78,3.14,5.41,9.48,3.65,6.21,3.41,9.45,...,1.48,4.28,2.99,1.42,1.09,5.78,5.84,4.91,NaN,NaN
2020-01-02 09:33:00,16.77,33.18,22.70,3.14,5.42,9.46,3.65,6.27,3.42,9.44,...,1.49,4.29,3.00,1.41,1.09,5.80,5.89,4.91,NaN,NaN
2020-01-02 09:34:00,16.80,33.44,22.49,3.14,5.42,9.50,3.64,6.26,3.42,9.45,...,1.49,4.28,3.00,1.41,1.09,5.81,5.98,4.91,NaN,NaN
2020-01-02 09:35:00,16.80,33.49,22.43,3.15,5.44,9.50,3.64,6.27,3.41,9.46,...,1.49,4.28,2.99,1.39,1.09,5.78,6.00,4.93,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-22 14:56:00,10.50,6.68,10.13,0.83,3.48,3.61,2.21,10.18,1.82,8.06,...,0.38,1.12,0.52,0.40,0.25,0.59,0.67,0.39,0.64,1.12
2024-04-22 14:57:00,10.51,6.69,10.10,0.83,3.48,3.61,2.21,10.19,1.82,8.06,...,0.38,1.12,0.52,0.40,0.25,0.59,0.67,0.39,0.64,1.12
2024-04-22 14:58:00,10.52,6.70,10.10,0.83,3.48,3.61,2.21,10.19,1.82,8.06,...,0.38,1.12,0.52,0.40,0.25,0.59,0.67,0.39,0.64,1.12
2024-04-22 14:59:00,10.52,6.70,10.10,0.83,3.48,3.61,2.21,10.19,1.82,8.06,...,0.38,1.12,0.52,0.40,0.25,0.59,0.67,0.39,0.64,1.12


In [ ]:
result = ff.read_min('000001.SZ')['close']
for i in list(data_all.columns):
    result = pd.concat([result,ff.read_min(i)['close']],axis = 1)

In [31]:
data_all = pd.read_pickle('/mydata2/wangs/data/fmins/close.pk')
data_all = data_all.loc[data_all.index[-21*240:-240]]
data_all

,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,...,600891.SH,600896.SH,600978.SH,601258.SH,601558.SH,603157.SH,603555.SH,603996.SH,688086.SH,688555.SH
2024-03-21 09:31:00,10.45,9.39,15.20,0.83,3.99,4.52,2.68,11.31,2.50,8.35,...,0.38,1.12,0.52,0.4,0.25,0.59,0.67,0.39,0.64,1.12
2024-03-21 09:32:00,10.45,9.40,15.15,0.83,4.02,4.51,2.68,11.31,2.50,8.37,...,0.38,1.12,0.52,0.4,0.25,0.59,0.67,0.39,0.64,1.12
2024-03-21 09:33:00,10.46,9.42,15.10,0.83,4.01,4.52,2.69,11.28,2.50,8.35,...,0.38,1.12,0.52,0.4,0.25,0.59,0.67,0.39,0.64,1.12
2024-03-21 09:34:00,10.46,9.40,15.03,0.83,4.01,4.53,2.69,11.28,2.50,8.35,...,0.38,1.12,0.52,0.4,0.25,0.59,0.67,0.39,0.64,1.12
2024-03-21 09:35:00,10.45,9.39,15.07,0.83,4.01,4.52,2.67,11.28,2.51,8.34,...,0.38,1.12,0.52,0.4,0.25,0.59,0.67,0.39,0.64,1.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-19 14:56:00,10.70,6.70,10.22,0.83,3.51,3.46,2.28,10.29,1.87,8.03,...,0.38,1.12,0.52,0.4,0.25,0.59,0.67,0.39,0.64,1.12
2024-04-19 14:57:00,10.69,6.70,10.21,0.83,3.51,3.46,2.28,10.30,1.87,8.03,...,0.38,1.12,0.52,0.4,0.25,0.59,0.67,0.39,0.64,1.12
2024-04-19 14:58:00,10.69,6.70,10.21,0.83,3.51,3.47,2.28,10.30,1.87,8.03,...,0.38,1.12,0.52,0.4,0.25,0.59,0.67,0.39,0.64,1.12
2024-04-19 14:59:00,10.69,6.70,10.21,0.83,3.51,3.47,2.28,10.30,1.87,8.03,...,0.38,1.12,0.52,0.4,0.25,0.59,0.67,0.39,0.64,1.12


In [34]:
test = ff.read_min('000001.SZ')
test.index[-21*240:-240]

Index(['2024-04-22 09:31:00', '2024-04-22 09:32:00', '2024-04-22 09:33:00',
       '2024-04-22 09:34:00', '2024-04-22 09:35:00', '2024-04-22 09:36:00',
       '2024-04-22 09:37:00', '2024-04-22 09:38:00', '2024-04-22 09:39:00',
       '2024-04-22 09:40:00',
       ...
       '2024-05-22 14:51:00', '2024-05-22 14:52:00', '2024-05-22 14:53:00',
       '2024-05-22 14:54:00', '2024-05-22 14:55:00', '2024-05-22 14:56:00',
       '2024-05-22 14:57:00', '2024-05-22 14:58:00', '2024-05-22 14:59:00',
       '2024-05-22 15:00:00'],
      dtype='object', length=4800)

In [55]:
result = pd.Series(np.nan,test.index[-21*240:-240])
for i in list(data_all.columns):
    series = pd.Series(ff.read_min(i)['close'],test.index[-21*240:-240])
    result = pd.concat([result,series],axis = 1)

In [50]:
pd.Series(ff.read_min('000005.SZ')['close'],test.index[-21*240:-240])

2024-04-22 09:31:00    0.83
2024-04-22 09:32:00    0.83
2024-04-22 09:33:00    0.83
2024-04-22 09:34:00    0.83
2024-04-22 09:35:00    0.83
                       ... 
2024-05-22 14:56:00     NaN
2024-05-22 14:57:00     NaN
2024-05-22 14:58:00     NaN
2024-05-22 14:59:00     NaN
2024-05-22 15:00:00     NaN
Name: close, Length: 4800, dtype: float64

In [49]:
ff.read_min('000005.SZ')['close']

2020-01-02 09:31:00    3.13
2020-01-02 09:32:00    3.14
2020-01-02 09:33:00    3.14
2020-01-02 09:34:00    3.14
2020-01-02 09:35:00    3.15
                       ... 
2024-04-25 14:56:00    0.83
2024-04-25 14:57:00    0.83
2024-04-25 14:58:00    0.83
2024-04-25 14:59:00    0.83
2024-04-25 15:00:00    0.83
Name: close, Length: 250800, dtype: float64

In [53]:
result

,0,close,close,close,close,close,close,close,close,close,...,close,close,close,close,close,close,close,close,close,close
2024-04-22 09:31:00,NaN,10.78,6.77,10.17,0.83,3.52,3.61,2.26,10.28,1.87,...,17.38,1.22,6.00,2.91,2.36,6.65,49.27,9.06,12.89,5.67
2024-04-22 09:32:00,NaN,10.79,6.82,10.21,0.83,3.51,3.63,2.23,10.24,1.86,...,17.42,1.23,5.99,2.93,2.36,6.56,49.80,9.06,13.11,5.67
2024-04-22 09:33:00,NaN,10.78,6.81,10.11,0.83,3.50,3.63,2.22,10.20,1.84,...,17.26,1.22,6.01,2.94,2.36,6.50,49.44,9.04,12.85,5.64
2024-04-22 09:34:00,NaN,10.74,6.79,10.02,0.83,3.49,3.63,2.21,10.19,1.83,...,17.32,1.20,6.01,2.92,2.34,6.41,49.32,9.01,12.93,5.61
2024-04-22 09:35:00,NaN,10.69,6.77,9.97,0.83,3.48,3.62,2.21,10.18,1.82,...,17.35,1.20,6.04,2.92,2.34,6.45,49.24,9.01,12.43,5.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-22 14:56:00,NaN,11.55,9.40,11.59,NaN,4.24,4.62,2.24,10.37,1.98,...,17.95,1.29,6.13,3.26,2.94,6.82,57.93,8.71,13.23,5.62
2024-05-22 14:57:00,NaN,11.55,9.40,11.59,NaN,4.24,4.63,2.23,10.37,1.97,...,17.95,1.30,6.14,3.26,2.94,6.83,57.90,8.71,13.27,5.61
2024-05-22 14:58:00,NaN,11.55,9.40,11.60,NaN,4.24,4.63,2.24,10.37,1.97,...,17.95,1.30,6.14,3.26,2.95,6.83,57.91,8.70,13.28,5.61
2024-05-22 14:59:00,NaN,11.55,9.40,11.60,NaN,4.24,4.63,2.24,10.37,1.97,...,17.95,1.30,6.14,3.26,2.95,6.83,57.91,8.70,13.28,5.61


In [70]:
result2 = result.iloc[:,1:]
result2.columns = data_all.columns

In [119]:
ff.read('synergy')

,20200102,20200103,20200106,20200107,20200108,20200109,20200110,20200113,20200114,20200115,...,20240408,20240409,20240410,20240411,20240412,20240415,20240416,20240417,20240418,20240419
000001.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.284755,0.281048,0.266407,0.279347,0.272885,0.267183,0.264040,0.260250,0.258510,0.279172
000002.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.341264,0.341944,0.332415,0.329310,0.320903,0.322821,0.322309,0.322644,0.329383,0.338925
000004.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.295437,0.297834,0.295403,0.301065,0.303591,0.315650,0.326367,0.338461,0.340177,0.341154
000005.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000006.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.265308,0.270457,0.264737,0.280409,0.277310,0.290069,0.289962,0.294087,0.301289,0.300946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603157.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
603555.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
603996.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
688086.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [122]:
data_all = pd.read_pickle('/home/wangs/data/fmins/close.pk')
data_all = data_all.loc[data_all.index[:-240]]

In [123]:
data_all

,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,...,600891.SH,600896.SH,600978.SH,601258.SH,601558.SH,603157.SH,603555.SH,603996.SH,688086.SH,688555.SH
2020-01-02 09:31:00,16.70,33.01,22.85,3.13,5.42,9.55,3.66,6.26,3.40,9.49,...,1.49,4.28,3.00,1.42,1.10,5.80,5.91,4.91,NaN,NaN
2020-01-02 09:32:00,16.83,33.10,22.78,3.14,5.41,9.48,3.65,6.21,3.41,9.45,...,1.48,4.28,2.99,1.42,1.09,5.78,5.84,4.91,NaN,NaN
2020-01-02 09:33:00,16.77,33.18,22.70,3.14,5.42,9.46,3.65,6.27,3.42,9.44,...,1.49,4.29,3.00,1.41,1.09,5.80,5.89,4.91,NaN,NaN
2020-01-02 09:34:00,16.80,33.44,22.49,3.14,5.42,9.50,3.64,6.26,3.42,9.45,...,1.49,4.28,3.00,1.41,1.09,5.81,5.98,4.91,NaN,NaN
2020-01-02 09:35:00,16.80,33.49,22.43,3.15,5.44,9.50,3.64,6.27,3.41,9.46,...,1.49,4.28,2.99,1.39,1.09,5.78,6.00,4.93,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-19 14:56:00,10.70,6.70,10.22,0.83,3.51,3.46,2.28,10.29,1.87,8.03,...,0.38,1.12,0.52,0.40,0.25,0.59,0.67,0.39,0.64,1.12
2024-04-19 14:57:00,10.69,6.70,10.21,0.83,3.51,3.46,2.28,10.30,1.87,8.03,...,0.38,1.12,0.52,0.40,0.25,0.59,0.67,0.39,0.64,1.12
2024-04-19 14:58:00,10.69,6.70,10.21,0.83,3.51,3.47,2.28,10.30,1.87,8.03,...,0.38,1.12,0.52,0.40,0.25,0.59,0.67,0.39,0.64,1.12
2024-04-19 14:59:00,10.69,6.70,10.21,0.83,3.51,3.47,2.28,10.30,1.87,8.03,...,0.38,1.12,0.52,0.40,0.25,0.59,0.67,0.39,0.64,1.12


In [128]:
ff.read('synergy')

,20200102,20200103,20200106,20200107,20200108,20200109,20200110,20200113,20200114,20200115,...,20240409,20240410,20240411,20240412,20240415,20240416,20240417,20240418,20240419,20240422
000001.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.281048,0.266407,0.279347,0.272885,0.267183,0.264040,0.260250,0.258510,0.279172,0.285189
000002.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.341944,0.332415,0.329310,0.320903,0.322821,0.322309,0.322644,0.329383,0.338925,0.339931
000004.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.297834,0.295403,0.301065,0.303591,0.315650,0.326367,0.338461,0.340177,0.341154,0.341908
000005.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000006.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.270457,0.264737,0.280409,0.277310,0.290069,0.289962,0.294087,0.301289,0.300946,0.300096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603157.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
603555.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
603996.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
688086.SH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
